In [18]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Assigment 2 Question 3").getOrCreate()
df = spark.read.option('header','true').csv("./data/TA_restaurants_curated_cleaned.csv",inferSchema=True)

In [19]:
df = df.withColumn("Reviews",F.expr("substring(Reviews,4,length(Reviews)-6)"))

In [20]:
split_cols = F.split(df["Reviews"],'\\], \\[')

df = df.withColumn('review',split_cols.getItem(0)).withColumn('date',split_cols.getItem(1))

In [21]:
df = df.withColumn('review',F.split(F.col('review'),","))
df = df.withColumn('date',F.split(F.col('date'),","))

print(df.printSchema())
df = df.withColumn("new",F.arrays_zip("review","date"))\
    .withColumn("new",F.explode("new"))\
        .select("ID_TA",F.col("new.review").alias("review"),F.col("new.date").alias("date"))
df = df.withColumn('review',F.regexp_replace('review',"'",''))
df = df.withColumn('date',F.regexp_replace('date',"'",''))
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Cuisine Style: string (nullable = true)
 |-- Ranking: double (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Price Range: string (nullable = true)
 |-- Number of Reviews: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- URL_TA: string (nullable = true)
 |-- ID_TA: string (nullable = true)
 |-- review: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date: array (nullable = true)
 |    |-- element: string (containsNull = true)

None
root
 |-- ID_TA: string (nullable = true)
 |-- review: string (nullable = true)
 |-- date: string (nullable = true)



In [22]:
df.toPandas().to_csv("output/modifiedData3.csv")